In [2]:
ollama.pull('llama3.1')

{'status': 'success'}

In [1]:
import pandas as pd
import ollama
import re

In [3]:
#pip install ollama

In [15]:
df1= pd.read_excel(r'/data/keywords092024.xlsx')
df1.head()

,keyword,totalcount
0,adventskalender,514036
1,Adventskalender,156879
2,adventskalender fÃ¼r frauen,140504
3,charlotte tilbury,134658
4,sol de janeiro,101680


In [17]:
df1 = df1.dropna(subset=['keyword'])

In [18]:
def classify_keyword(keyword):
    content = f'Classify the following Keyword: {keyword} into the appropriate category, provide the output in the following format:Category: [Category]. Categories are Skincare, Haircare, Hair Color, Eye Makeup, Lip Makeup, Face Makeup, Nails, Body, Women Fragrances, Men-/Unisex Fragrances, Beauty Tools&Devices, Accessories, Wellness&Supplements, Offers&Promotions, Other'
    response = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'user',
            'content': content,
        },
    ])
    return response['message']['content']



In [20]:
#categories = []
#d2 = pd.DataFrame(columns=['category'])

In [21]:
# Function to parse the output from ollama chat
def parse_category(response):
    match = re.search(r'Category:\s*(.*?)\s*$', response, re.IGNORECASE) # regex to extract the keyword and category
    if match:
        return match.group(1).strip()
    else:
        return None

In [22]:
for index, row in df1.iterrows():
    try:
        response = classify_keyword(row['keyword'])
        category = parse_category(response)
        categories.append(category)  # Append the parsed category
    except Exception as e:
        print(f"Error processing keyword '{row['keyword']}': {e}")
        categories.append(None)  # Preserve list length with None

In [23]:
print(f"Original DataFrame length: {len(df1)}")
print(f"Categories list length: {len(categories)}")

Original DataFrame length: 200
Categories list length: 200


In [24]:
if len(categories) == len(df1):
   df1['category'] = categories
else:
    raise ValueError("Mismatch in lengths!")


In [25]:
df1.head()

,keyword,totalcount,category
0,adventskalender,514036,Other.
1,Adventskalender,156879,None
2,adventskalender fÃ¼r frauen,140504,None
3,charlotte tilbury,134658,None
4,sol de janeiro,101680,Women Fragrances
